In [96]:
import os, re, urllib2, xml.dom

In [145]:
def named_node_map_to_dict(nnm):
    ret = {}
    for i in range(nnm.length):
        ret[nnm.item(i).name] = nnm.item(i).value
    return ret

def alternate_id(id):
    if re.match(r'\w\d\d\d\w0\d\d\d', id):
        return id[:-4] + id[-3:]
    if re.match(r'\w\d\d\d\d\d\d', id):
        return id[:-3] + '0' + id[-3:]

#print alternate_id('P000P0000')
#print alternate_id('P038014')

def find_id_year(id, year):
    path = 'columncache/census{year}_block2010/{id}.numpy'.format(**locals())
    if os.path.exists(path):
        return path
    if alternate_id(id):
        bad_id = alternate_id(id)
        bad_path = 'columncache/census{year}_block2010/{bad_id}.numpy'.format(**locals())
        if os.path.exists(bad_path):
            target = os.path.basename(bad_path)
            print 'Running os.symlink({target}, {path})'.format(**locals())
            os.symlink(target, path)
        return path
    raise Exception('Cannot find original for {path}'.format(**locals()))

In [162]:
def write_description(year, doc_url):
    xmlurl = "https://api.census.gov/data/{year}/sf1/variables.xml".format(**locals())
    xmldoc = xml.dom.minidom.parseString(urllib2.urlopen(xmlurl).read())

    cols = {}

    for var in xmldoc.getElementsByTagName('var'):
        parsed = named_node_map_to_dict(var.attributes)
        id = parsed['xml:id']
        if re.match(r'[HP]\d', id):
            find_id_year(id, year)
            if not 'concept' in parsed:
                # Patch a couple bad records from US Census
                if year == '2000' and id == 'P027D004':
                    parsed['concept'] = 'P27D. Relationship By Household Type (Including Living Alone) (Asian Alone)'
                elif year == '2000' and id == 'H017069':
                    parsed['concept'] = 'H17. Tenure By Household Type (Including Living Alone) By Age Of Householder'
            name = re.sub(r'\s+\[\d+\]', '', parsed['concept']) + ' &mdash; '
            label = parsed['label']
            label = re.sub(r':? !! ', ' &mdash; ', label)
            label = re.sub(r'\s*:\s*', ' &mdash; ', label)
            label = re.sub(r'&mdash;\s+$', '', label)
            label = label.replace("'&'", '&')
            label = label.replace("'<'", '<')
            label = label.replace("'>'", '>')
            name += label
            cols[id] = name

    html = []
    html += '<br><a href="{doc_url}">Link to US Census technical documentation (PDF)</a><br><br>\n'.format(**locals())
    html += '<table>\n'

    for id in sorted(cols.keys()):
        name = cols[id]
        html += '<tr><td>{id}</td><td>{name}</td></tr>\n'.format(**locals())

    html += '</table>\n'

    dest = 'columncache/census{year}_block2010/description.html'.format(**locals())
    open(dest, 'w').write(''.join(html))

In [165]:
write_description('1990', 'https://www2.census.gov/prod2/decennial/documents/D1-D90-S101-14-TECH-01.pdf')
write_description('2000', 'https://www.census.gov/prod/cen2000/doc/sf1.pdf')
write_description('2010', 'https://www2.census.gov/library/publications/cen2010/doc/sf1.pdf')